In [1]:
import os
import psycopg2
from dotenv import load_dotenv
from typing import Any, List, Tuple, Optional

load_dotenv()

True

In [2]:
class SQLWrapper:
    def __init__(self, db_uri_env_var: str = "DATABASE_URI_CONTATOS"):
        """
        Initialize the SQLWrapper with the database URI from environment variables.

        Args:
            db_uri_env_var (str): The environment variable name for the database URI.
        """
        load_dotenv()
        self.db_uri = os.getenv(db_uri_env_var)
        if not self.db_uri:
            raise ValueError(
                f"Database URI for {db_uri_env_var} not found in environment variables."
            )

    def _connect(self) -> psycopg2.extensions.connection:
        """
        Establish a connection to the database.

        Returns:
            psycopg2.extensions.connection: The database connection object.
        """
        return psycopg2.connect(self.db_uri)

    def execute_query(
        self,
        query: str,
        params: Optional[Tuple[Any, ...]] = None,
    ) -> List[Tuple[Any, ...]]:
        """
        Execute a SQL query and return the results. Commit the changes for all operations.

        Args:
            query (str): The SQL query to execute.
            params (Optional[Tuple[Any, ...]]): The parameters to pass to the query.

        Returns:
            List[Tuple[Any, ...]]: The results of the query.

        Raises:
            Exception: If the query execution fails.
        """
        try:
            connection = self._connect()
            cursor = connection.cursor()
            cursor.execute(query, params)

            results = cursor.fetchall() if cursor.description else []
            connection.commit()

            cursor.close()
            connection.close()
            return results
        except Exception as e:
            raise Exception(f"Failed to execute query. Error: {str(e)}")

    def fetch_table_as_dict(self, table_name: str) -> dict:
        """
        Fetch data from a specified table and return it as a dictionary of lists.

        Args:
            table_name (str): The name of the table to fetch data from.

        Returns:
            dict: A dictionary where each key is a column name and each value is a list of column values.
        """
        columns_query = f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}';"
        columns = [desc[0] for desc in self.execute_query(columns_query)]
        data_query = f"SELECT * FROM {table_name};"
        raw_data = self.execute_query(data_query)
        data_dict = {col: [] for col in columns}

        for row in raw_data:
            for col, val in zip(columns, row):
                data_dict[col].append(val)

        return data_dict

    def list_tables(self) -> List[str]:
        """
        List all tables in the public schema.

        Returns:
            List[str]: A list of table names.
        """
        query = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"
        return [table[0] for table in self.execute_query(query)]

In [3]:
wrapper = SQLWrapper()

In [4]:
wrapper.list_tables()

['bots_de_atendimento',
 'contatos_filipe',
 'numeros_telera_cogmo',
 'numeros_secretarias_cogmo',
 'grupos_de_whatsapp']

In [5]:
warmup_numbers_table = "numeros_telera_cogmo"
warmup_numbers_dict = wrapper.fetch_table_as_dict(warmup_numbers_table)
warmup_numbers_dict

{'id': [2, 3, 1, 4],
 'updated_at': [None, None, None, None],
 'ddd': [datetime.datetime(2024, 12, 11, 20, 39, 54),
  datetime.datetime(2024, 12, 11, 20, 40, 1),
  datetime.datetime(2024, 12, 11, 20, 28, 17),
  datetime.datetime(2024, 12, 11, 20, 51, 47)],
 'telefone': [None, None, datetime.datetime(2024, 12, 11, 20, 43, 47), None],
 'created_at': ['us516jtuyxw461hz',
  'us516jtuyxw461hz',
  'us516jtuyxw461hz',
  'uskm9u2taom7mlwq'],
 'nome': [None, None, 'us516jtuyxw461hz', None],
 'created_by': [None, None, None, None],
 'updated_by': [None, None, None, None]}

In [6]:
contact_numbers_tables = ["bots_de_atendimento", "contatos_filipe"]
contact_numbers_dict = {table: wrapper.fetch_table_as_dict(table) for table in contact_numbers_tables}
contact_numbers_dict

{'bots_de_atendimento': {'updated_at': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
  'telefone': ['Banco do Brasil',
   'Samsung',
   'Claro',
   'NET',
   'Tim',
   'Oi (Atendimento IA)',
   'Oi (Atendimento Convencional)',
   'Vivo',
   'Globoplay',
   'Caixa Econômica Federal',
   'Bradesco Seguros',
   None],
  'ddd': [datetime.datetime(2024, 11, 11, 19, 19, 32),
   datetime.datetime(2024, 11, 11, 19, 20, 21),
   datetime.datetime(2024, 11, 11, 19, 21, 25),
   datetime.datetime(2024, 11, 11, 19, 22, 11),
   datetime.datetime(2024, 11, 11, 19, 22, 59),
   datetime.datetime(2024, 11, 11, 19, 24, 12),
   datetime.datetime(2024, 11, 11, 19, 25, 18),
   datetime.datetime(2024, 11, 11, 19, 25, 54),
   datetime.datetime(2024, 11, 11, 19, 28, 15),
   datetime.datetime(2024, 11, 11, 19, 29, 53),
   datetime.datetime(2024, 11, 11, 19, 31, 22),
   datetime.datetime(2024, 11, 11, 19, 32, 17)],
  'id': [datetime.datetime(2024, 11, 11, 19, 45, 47),
   datetime.datetime(2024, 11, 11, 19, 21, 24),
  

In [9]:
def distribute_contacts(warmup_numbers_dict: dict, contact_numbers_dict: dict) -> dict:
    """
    Distribute contacts from contact_numbers_dict among the entries in warmup_numbers_dict.

    Args:
        warmup_numbers_dict (dict): Dictionary containing support agents' phone numbers and metadata.
        contact_numbers_dict (dict): Dictionary containing customer contacts to be distributed.

    Returns:
        dict: Updated warmup_numbers_dict with contacts distributed among the agents.
    """
    # Initialize the 'contacts' key for each agent
    num_agents = len(warmup_numbers_dict['id'])
    warmup_numbers_dict['contacts'] = [[] for _ in range(num_agents)]

    # Flatten the contact numbers into a single list
    all_contacts = []
    for table in contact_numbers_dict.values():
        for i in range(len(next(iter(table.values())))):  # Assuming all columns have the same length
            contact = {key: table[key][i] for key in table}
            all_contacts.append(contact)

    # Calculate the number of contacts per agent
    contacts_per_agent = len(all_contacts) // num_agents
    extra_contacts = len(all_contacts) % num_agents

    # Distribute contacts among agents
    start_index = 0
    for i in range(num_agents):
        end_index = start_index + contacts_per_agent + (1 if i < extra_contacts else 0)
        warmup_numbers_dict['contacts'][i].extend(all_contacts[start_index:end_index])
        start_index = end_index

    return warmup_numbers_dict

# Example usage
warmup_numbers_dict = distribute_contacts(warmup_numbers_dict, contact_numbers_dict)
warmup_numbers_dict



{'id': [2, 3, 1, 4],
 'updated_at': [None, None, None, None],
 'ddd': [datetime.datetime(2024, 12, 11, 20, 39, 54),
  datetime.datetime(2024, 12, 11, 20, 40, 1),
  datetime.datetime(2024, 12, 11, 20, 28, 17),
  datetime.datetime(2024, 12, 11, 20, 51, 47)],
 'telefone': [None, None, datetime.datetime(2024, 12, 11, 20, 43, 47), None],
 'created_at': ['us516jtuyxw461hz',
  'us516jtuyxw461hz',
  'us516jtuyxw461hz',
  'uskm9u2taom7mlwq'],
 'nome': [None, None, 'us516jtuyxw461hz', None],
 'created_by': [None, None, None, None],
 'updated_by': [None, None, None, None],
 'contacts': [[{'updated_at': 1,
    'telefone': 'Banco do Brasil',
    'ddd': datetime.datetime(2024, 11, 11, 19, 19, 32),
    'id': datetime.datetime(2024, 11, 11, 19, 45, 47),
    'created_at': 'us9rgttbuqj7xg7n',
    'remotejid': 'us9rgttbuqj7xg7n',
    'nome': 40040001,
    'created_by': 61,
    'updated_by': '556140040001@s.whatsapp.net'},
   {'updated_at': 2,
    'telefone': 'Samsung',
    'ddd': datetime.datetime(2024, 

In [ ]:
import pandas as pd
import numpy as np

# Load data
agents = pd.read_csv('numeros_telera_cogmo_exported_1.csv')
bots = pd.read_csv('bots_de_atendimento_exported_1.csv')
contacts = pd.read_csv('contatos_filipe_exported_1.csv')

# Combine both contact lists
all_customers = pd.concat([
    bots[['nome', 'ddd', 'telefone']],
    contacts[['nome', 'ddd', 'telefone']]
]).reset_index(drop=True)

# Create final dataframe by randomly assigning agents
num_customers = len(all_customers)
agent_indices = np.random.randint(0, len(agents), size=num_customers)

result = pd.DataFrame({
    'agente_cogmo_ddd': agents.iloc[agent_indices]['ddd'].values,
    'agente_cogmo_telefone': agents.iloc[agent_indices]['telefone'].values,
    'customer_name': all_customers['nome'],
    'customer_ddd': all_customers['ddd'],
    'customer_telefone': all_customers['telefone']
})

# Save to CSV
result.to_csv('distribuicao_contatos.csv', index=False)

# Print summary
print(f"Total contacts distributed: {len(result)}")
print("\nContacts per agent:")
print(result.groupby(['agente_cogmo_ddd', 'agente_cogmo_telefone']).size())